# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
import PyPDF2 as ppdf
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','AllRatMaxPos.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

Plot the definition of before after ...

In [ ]:
def plot_session_def(ax, sessionSlices= tuple()):
    eps=0
    xpos=ax.get_xticks()
    
    for sli in sessionSlices:
        assert isinstance(sli, slice), 'Bad Boy!'        
        stop=sli.stop
        if stop is None:
            stop=-1
        start=sli.start
        if start>=0:
            start+=1
#             stop+=1
        
        ax.axvspan(xmin=start+eps, xmax=stop-eps, zorder=-1, alpha=.8, ec= None,
                  color='xkcd:ivory')
        
    if len(sessionSlices)==3:
        for sli,String in zip(sessionSlices,['Before','Acute','Stable']):
            stop=sli.stop
            if stop is None:
                stop=-1
            start=sli.start
            if start>=0:
                start+=1
            
            mid=(start+stop)/2
            
            ax.text(mid,ax.get_ylim()[1],s=String,
                   fontsize='xx-small', ha='center',va='top')

---

plotting group time course

In [ ]:
def normalize_performance(a,nPre):
    """implement a normalizing function to operate on 'a' and nPre first point are baseline
       a: np.array: animal x session
    """
    meanBase= np.nanmedian(a[:,:nPre], axis=1, keepdims=True)
    
    out=a-meanBase
    return out

def plot_normalized_time_course(root, ax, Profiles, Animals=None, TaskParamToPlot='% good trials', shift=0,
                   nPre=5, nPost=15, **kwarg):

    #getting the data
    animalList,SessionDict=event_detect(root, Profiles[0], Profiles[1])
    
    if Animals is None:
        Animals=animalList
    
    SessionDict={animal:SessionDict[animal] for animal in Animals}
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])
    
    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values()))
    
    xData=np.append(np.arange(-nPre,0),np.arange(1,nPost+1))
    data=data[:,nSessionPre-nPre:nSessionPre+nPost]
    
    data=normalize_performance(data,nPre)
    
    reliableAnimals=np.sum(np.isnan(data),axis=1) < data.shape[1] / 2
    
    data=data[reliableAnimals,:]
    animalList=np.array(list(Results[TaskParamToPlot].keys()))[reliableAnimals]
    
    groupData=np.nanmedian(data,axis=0)
    
    groupErr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(xData + shift, groupData,
                abs(groupErr-groupData),
                fmt='-o', zorder=5, ms=2, elinewidth=1,alpha=.8, **kwarg)
    
    ax.axhline(0, ls=':',c='gray',lw=1)
    
    def _tik(x,pos):
        if x in [-nPre,-1,1,nPost]:
            return ('$%+g$' if x>0 else '$%g$')%x
        else:
            return ''
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(_tik))
    
    ax.set_xlim([-nPre-.3,nPost+.3])
    ax.spines['bottom'].set_bounds(-nPre,nPost)
    ax.set_xticks(xData)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
#     ax.set_ylim([0,100])
    ax.set_ylabel('Norm. '+TaskParamToPlot)
    ax.set_xlabel('Session # relative to lesion')

    return data,animalList

def add_legend_for_rat_groups(ax,N):
    g_marker = matplotlib.lines.Line2D([0,0], [1,1], color='gray',lw=1,
                                       markeredgecolor='gray', marker='o',
                                       markerfacecolor='gray', markersize=2, label=f'$\Delta$Speed$>0$\n($n={N[0]}$)')
    b_marker = matplotlib.lines.Line2D([0,0], [1,1], color='k',
                                       markeredgecolor='k', marker='o',lw=1,
                                       markerfacecolor='k', markersize=2, label=f'$\Delta$Speed$<0$\n($n={N[1]}$)')


    leg=ax.legend(handles=[b_marker,g_marker],loc=(0,.9),ncol=2,
                  facecolor=None,edgecolor=None, fontsize=5,frameon=False)
    return leg

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    profile2={'Type': ['Good'],
             'rewardType': ['Progressive'],
             'option': ['not used', 'AsymmetricLesion'],
             'initialSpeed': ['0', '10'],
             'Speed': ['10'],
             'Tag': ['Late-Lesion_DLS',
              'Late-Lesion_DMS-Sharp',
              'Late-Lesion_DMS',
              'Late-Lesion_DS',
              'Late-Lesion_DS-Sharp']}

    #number of sessions to plot
    animalList,_=event_detect(root,profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

#     a=plot_normalized_time_course(root, ax, Profiles,Animals=animalList,TaskParamToPlot=TaskParamToPlot,color='k',shift=-.1*.25)
    add_legend_for_rat_groups(ax,(5,1))

---

plot the speed correlation plot

In [ ]:
def _late_lesion_effect(root, Profiles, badAnimals, TaskParamToPlot:str,
                        preSlice=slice(-5,None), postSlice=slice(0,5)):
    
    _,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    [sessionDict.pop(key, None) for key in badAnimals]
    
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                 sessionDict,
                                                 parameter=param,
                                                 redo=False,
                                                 TaskParamToPlot=[TaskParamToPlot])

    data=np.array(list(Results[TaskParamToPlot].values()))
    slicedPreData  = np.ones((data.shape))*np.nan
    slicedPostData = np.ones((data.shape))*np.nan
    
    for row,Dnan in enumerate(data):
        Dpre= Dnan[:nSessionPre]#[~ np.isnan(Dnan[:nSessionPre])]
        Dpost=Dnan[nSessionPre:]#[~ np.isnan(Dnan[nSessionPre:])]
        preData =Dpre[preSlice]
        postData=Dpost[postSlice]

        slicedPreData[row,:len(preData)]=preData
        slicedPostData[row,:len(postData)]=postData
    
    yPre =np.nanmean(slicedPreData ,axis=1)
    yPost=np.nanmean(slicedPostData,axis=1)

    behav=[]
    animals=[]
    for i,animal in enumerate(Results[TaskParamToPlot].keys()):
        if np.isnan(yPre[i]) or np.isnan(yPost[i]):
            logging.error(f'{animal}: {TaskParamToPlot} not defined!')
            continue

        behav.append(yPost[i]-yPre[i])
        animals.append(animal)
    
    return behav, animals

def late_lesion_correlation_with_size(root, ax, Profiles, Animals, color, TaskParamToPlot:str,
                                      preSlice=slice(-5,None), postSlice=slice(0,5), Excluded='RatBAD'):
    
    _,sessionDict=event_detect(root, Profiles[0], Profiles[1])
    
    sessionDict={animal:sessionDict[animal] for animal in Animals}
    
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                 sessionDict,
                                                 parameter=param,
                                                 redo=False,
                                                 TaskParamToPlot=[TaskParamToPlot])

    data=np.array(list(Results[TaskParamToPlot].values()))
    slicedPreData  = np.ones((data.shape))*np.nan
    slicedPostData = np.ones((data.shape))*np.nan
    
    for row,Dnan in enumerate(data):
        Dpre= Dnan[:nSessionPre]#[~ np.isnan(Dnan[:nSessionPre])]
        Dpost=Dnan[nSessionPre:]#[~ np.isnan(Dnan[nSessionPre:])]
        preData =Dpre[preSlice]
        postData=Dpost[postSlice]

        slicedPreData[row,:len(preData)]=preData
        slicedPostData[row,:len(postData)]=postData
    
    yPre =np.nanmean(slicedPreData ,axis=1)
    yPost=np.nanmean(slicedPostData,axis=1)

    size=[]
    behav=[]
    animals=[]
    for i,animal in enumerate(Results[TaskParamToPlot].keys()):
        if np.isnan(yPre[i]) or np.isnan(yPost[i]):
            logging.error(f'{animal}: {TaskParamToPlot} not defined!')
            continue
        try:
            Hist=HistologyExcel('/NAS02',animal)
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        
        behav.append(yPost[i]-yPre[i])
        size.append(Hist.lesion_size())
        animals.append(animal)
        
    
    #plotting
    if ax is not None:
        for i,animal in enumerate(animals):
            if animal == Excluded: continue
            _,tag=lesion_type(root,animal)
            ax.scatter(size[i], behav[i], s=5, c=color[tag]);
#             ax.annotate(animal[-3:], (size[i],behav[i]), fontsize='xx-small')

        ax.set_xticks(np.arange(0,1.01,.1))
        ax.set_xticklabels(['0','','','','','0.5','','','','','1'])
        ax.spines['bottom'].set_bounds(0,1)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlim([-.02,1.02])
        ax.set_ylabel(f'$\Delta$ {TaskParamToPlot}')
        ax.set_xlabel('Lesion size')

    
    return behav, size, animals

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    Profiles=(profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    preSlice=slice(-5,None)
    postSlice=slice(3,8)
    
    color={'DS':'r','DMS':'g', 'DLS':'b', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    
#     a=late_lesion_correlation_with_size(root, ax=ax, Profiles=Profiles,color=color, TaskParamToPlot=TaskParamToPlot,
#                                       preSlice=preSlice, postSlice=postSlice)

---

plotting group double errorbars for speed 

In [ ]:
def plot_event_1on1(root, ax, profile, color, TaskParamToPlot='% good trials',
                   x_pos=1, nSlice=slice(-5,None), seed=1, animal_plot=True):
    
    animalList=batch_get_animal_list(root,profile)
    
    data=[]
    for animal in animalList:
        out=animal_learning_stats(root, animal, TaskParamToPlot, profile, nSlice)
        if len(out)==0:
            continue
        data.append(out)

    
    data=np.array(data)
    data=np.nanmedian(data,axis=1)
    data=data[np.logical_not(np.isnan(data))]
    
    #plotting individual animals
    if animal_plot:
        ax.boxplot(x=data,whis=[5,95],
              positions=[x_pos], widths=.3,
              showcaps=False, showfliers=False,
              medianprops={'color':color}, boxprops={'color':color}, whiskerprops={'color':color},
              zorder=1)
        
        
#     ax.set_xlim([x_pos[0]-diff/8,x_pos[-1]+diff/8])
#     ax.set_xticks(x_pos)
#     ax.set_xticklabels(['Before','Stable'])
#     ax.xaxis.set_tick_params(rotation=70)
#     ax.spines['bottom'].set_bounds(x_pos[0],x_pos[-1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
#     ax.set_ylabel(TaskParamToPlot)
#     ax.set_xlabel('Sessions relative to lesion')

    return data

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
                     'rewardType':'Progressive',
                     'initialSpeed':['10'],
                     'Speed':'10',
                     'Tag':['Control', 'Control-Sharp']
                      }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DMS-Sharp','Late-Lesion_DLS','Late-Lesion_DS-Sharp']
             }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot="Front Starting Trials"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    ax.set_xticks([])
    data=plot_event_1on1(root, ax, profile1, color='k', TaskParamToPlot="Maximum Position",
                   x_pos=1, nSlice=slice(25,30), seed=1, animal_plot=True)
    
    

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='k'
    DLSColor='xkcd:orange'
    DMSColor='purple'
    DSColor='xkcd:green'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    colorSig='goldenrod'
    
    
    #===============================================
    
    # GRID 5 PARAMS
    
    TaskParamToPlot5="standard deviation of entrance time"
    nPre5=5
    nPost5=10
    _s=.15
    ShiftDLS5 =-_s
    ShiftDMS5 =0*_s
    ShiftDS5  =_s
    
    
    
    profilePreLesion5={'Type':'Good',
                     'rewardType':'Progressive',
                     'initialSpeed':['10'],
                     'Speed':'10',
                     'Tag':['Control', 'Control-Sharp']
                      }
    
    profileDLS5={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS5={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS5={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    ProfilesDLS5=(profilePreLesion5,profileDLS5,)
    ProfilesDMS5=(profilePreLesion5,profileDMS5,)
    ProfilesDS5 =(profilePreLesion5,profileDS5,)
    
    
    #================================================
    
    # GRID 6 PARAMS

    profileLesions6={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS5['Tag'], *profileDMS5['Tag'], *profileDS5['Tag'])))
                     }
    profileLesions6['Tag'].extend(['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS',
                                   'Early-Lesion_DS-NoHab','Early-Lesion_DMS-NoHab'])

    Profiles6=(profilePreLesion5,profileLesions6)
    preSlice6Ctrl=slice(25,30)
    preSlice6=slice(-5,None)
    TaskParamToPlot6="Maximum Position"

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params({'axes.labelsize':'x-small'})
    figsize=(3,1)
    fig=plt.figure(figsize=figsize,dpi=600)
    
   
    
    ##########################################
    # 5: Normalized Conditional Max Pos time course
    gs5= fig.add_gridspec(nrows=1, ncols=1, left=.0, bottom=0, right=.35, top=.4)
    ax5= fig.add_subplot(gs5[0])
    
    data5,goodAnimals5=plot_normalized_time_course(root, ax5, Profiles3,Animals=goodAnimals3,
                                                   TaskParamToPlot=TaskParamToPlot3,color='k')
    

    ax5.text(-5,-15,f'$n={data5.shape[0]}$ rats',
          ha='left',va='bottom',fontsize='xx-small',color='k')
    
    plot_session_def(ax5, sessionSlices= (slice(-2,None),slice(0,2)))
    plot_session_def(ax5, sessionSlices= (slice(13,15),))

    #STATS
    nPre=5
    a=np.nanmean(data5.T[nPre-2:nPre,:],axis=0)
    b=np.nanmean(data5.T[nPre:nPre+2,:],axis=0)
    _,s=bootstrapTest(a-b,10000)
    ax5.text(1.5,5,s,fontsize=5,c='k',ha='center')
    
    a=np.nanmean(data5.T[nPre-2:nPre,:],axis=0)
    b=np.nanmean(data5.T[nPre+13:,:],axis=0)
    _,s=bootstrapTest(a-b,10000)
    ax5.text(14.5,5,s,fontsize=5,c='k',ha='center')
    

    
    ax5.set_ylim([-15,5])
    ax5.set_yticks(range(-15,6,5))
    ax5.set_ylabel('Norm. Max. Pos. (cm)',labelpad=0)
    ax5.spines['left'].set_bounds(-15,5)
    

  
    ##########################################
    # 4: % Max Pis correlation
    gs4= fig.add_gridspec(nrows=1, ncols=1, left=0.48, bottom=0, right=.72, top=.35)
    ax4= fig.add_subplot(gs4[0])
  
    behav4, size4, animals4=late_lesion_correlation_with_size(root, ax=ax4, Profiles=Profiles3, Animals=goodAnimals5,
                                                              color=ColorCode, TaskParamToPlot=TaskParamToPlot3,
                                                              preSlice=preSlice3, postSlice=finSlice3,
                                                              Excluded=sessionBefore2[:6])
    
    
    r4,p4=stats.pearsonr(size4, behav4)
    s=f'$r=$'+'$%+.2f$'%(r4)+'\n'+'$p=$'+'{}'.format(SciNote(p4))
    ax4.text(1,10,s, ha='right',va='top', fontsize='xx-small')

    ax4.set_ylim([-20,10])
    ax4.spines['left'].set_bounds(-20,10)
    ax4.set_yticks([-20,-10,0,10])
    ax4.set_ylabel('$\Delta$Max. Pos. (cm)')
    
    
    
    ##########################################
    # 6: ALL rat comparison
    gs6= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=0, right=.45, top=1)
    ax6= fig.add_subplot(gs6[0])
    
    data6 =plot_event_1on1(root, ax6, Profiles6[0], color=CtrlColor, TaskParamToPlot=TaskParamToPlot6,
                   x_pos=0, nSlice=preSlice6Ctrl)
    data6_=plot_event_1on1(root, ax6, Profiles6[1], color=DMSColor, TaskParamToPlot=TaskParamToPlot6,
                   x_pos=1, nSlice=preSlice6)

    
    

    ax6.set_xlim([-.4,1.4])
    ax6.set_ylim([63,85])
    ax6.spines['left'].set_bounds(65,85)
    ax6.spines['bottom'].set_bounds(0,1)
    ax6.set_yticks(np.arange(65,86,5))
    ax6.set_xticks([0,1])
    ax6.set_xticklabels([f'Control\n$n={len(data6)}$',f'Lesion\n$n={len(data6_)}$'])
    ax6.set_ylabel('Max. Pos. (cm)')

    
        
#   #STATS
    test6=permtest(group1=data6, group2=data6_, nIterations=10000)
    s=f'$p={test6.pVal[0]}$' if test6.significantDiff[0] else 'n.s.'
    s='p<0.0001' if test6.pVal[0]==0 else s
    test6.plotSigPair(ax6,y=84.5,x=(0,1), s=s, color=colorSig,lw=.8)    
    
    
    
    

    
#     fig.align_ylabels([ax2,ax4,ax6])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(ax1,ax1_,ax1__l,ax1___r,axes2[0],axes3[0],axes4[0],ax5,ax7,ax11)
#     OFFX=np.array([.02]*len(AXES))
#     OFFY=np.array([.01]*len(AXES))

#     OFFX[[-3,-2,-1]]=0.06
#     OFFX[[3,2,1]]=0.04
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','AllRatMaxPos.pdf'),
                format='pdf', bbox_inches='tight')
    
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()